In [1]:
import pulp
import re

In [2]:
# Names for breweries
breweries = ["B1", "B2", "B3", "B4"]

# Dictionary for maximumn units of beer from each brewer
breweries_max = {"B1": 2000,
                   "B2": 2500,
                   "B3": 3500,
                   "B4": 2000}

breweries_min = {"B1": 100,
                       "B2": 150,
                       "B3": 200,
                       "B4": 100}

packaging_facilities = ["P1", "P2", "P3"]

packaging_maximum = {"P1":500,
              "P2":1500,
              "P3":2500}

 # Dictionary for minimum number of units packaging facilities would package
# Values at the low end would suggest closing the packaging facility
packaging_minimum = {"P1":50,
          "P2":100,
          "P3":150}


# Names of demand points (retailers and pubs)
demanding = ["D01", "D02", "D03", "D04", "D05",
             "D06", "D07", "D08", "D09", "D10",
             "D11", "D12", "D13", "D14", "D15"]

demand = {"D01": 48, "D02": 84, "D03": 64, "D04": 106, "D05": 47,
              "D06": 57, "D07": 64, "D08": 93, "D09": 74, "D10": 41,
              "D11": 61, "D12": 42, "D13": 57, "D14": 70, "D15": 41}

total_demand = sum(demand.values())

In [3]:
brew_to_pack_shipping_costs = [# packaging facilities in columns
         #  P1   P2   P3 Packaging Facilities in cols
         [1.55,0.51,0.90], # B1   Breweries in rows
         [0.81,3.18,0.65], # B2
         [2.13,0.97,0.51], # B3
         [1.23,2.15,2.08]  # B4
         ]

In [4]:
pack_to_demand_shipping_costs = [
        [4.82, 2.05, 4.42, 3.83, 0.97,
         3.04, 3.91, 4.03, 5.11, 0.90,
         4.39, 0.85, 2.81, 3.94, 1.04], # from packaging facility 1
        [1.83, 4.03, 3.95, 4.21, 4.78,
         3.20, 1.88, 2.96, 5.11, 2.67,
         4.14, 1.22, 5.10, 3.47, 1.92],# from packaging facility 2
        [2.66, 0.95, 3.94, 2.04, 2.35,
         1.42, 3.60, 3.17, 1.34, 4.51,
         0.74, 0.94, 1.98, 4.77, 2.04] # from packaging facility 3
        ]

In [5]:
first_costs = pulp.makeDict([breweries, packaging_facilities],brew_to_pack_shipping_costs,0)
second_costs = pulp.makeDict([packaging_facilities, demanding],pack_to_demand_shipping_costs,0)

In [6]:
prob = pulp.LpProblem("Beer_Distribution_Problem",pulp.LpMinimize)

In [7]:
first_routes = [(i,j) for i in breweries for j in packaging_facilities]

In [8]:
first_vars = pulp.LpVariable.dicts("route",(breweries,packaging_facilities),0,None,pulp.LpContinuous)

In [9]:
second_routes = [(j,k) for j in packaging_facilities for k in demanding]

In [10]:
second_vars = pulp.LpVariable.dicts("route", (packaging_facilities,demanding),0,None,pulp.LpContinuous)


In [11]:
#objective_function
prob += pulp.lpSum([first_vars[i][j]*first_costs[i][j] for (i,j) in first_routes]) + pulp.lpSum([second_vars[j][k]*second_costs[j][k] for (j,k) in second_routes])

In [12]:
#constraints for max production for breweries
for i in breweries:
    prob += pulp.lpSum([first_vars[i][j] for j in packaging_facilities]) <= breweries_max[i]

In [13]:
#constraints for minimum production for breweries
prob += pulp.lpSum([first_vars[i][j] for j in packaging_facilities]) >= breweries_min[i]

In [14]:
#constraints for max production at the packaging faciltiies
for j in packaging_facilities:
    prob += pulp.lpSum([second_vars[j][k] for k in demanding]) <= packaging_maximum[j]

In [15]:
#constraints for min production at packaging facilities
for j in packaging_facilities:
    prob += pulp.lpSum([second_vars[j][k] for k in demanding]) >= packaging_minimum[j]

In [16]:
#constraints for demand points
for k in demanding:
    prob += pulp.lpSum([second_vars[j][k] for j in packaging_facilities]) >= demand[k]

In [17]:
#constraint for packaging input being larger than packaging output
for j in packaging_facilities:
    prob += pulp.lpSum([first_vars[i][j] for i in breweries]) >= pulp.lpSum([second_vars[j][k] for k in demanding])

In [18]:
prob.solve()

1

In [19]:
for v in prob.variables():
    print(v.name, "=", round(v.varValue))

route_B1_P1 = 0
route_B1_P2 = 275
route_B1_P3 = 0
route_B2_P1 = 29
route_B2_P2 = 0
route_B2_P3 = 0
route_B3_P1 = 0
route_B3_P2 = 0
route_B3_P3 = 545
route_B4_P1 = 100
route_B4_P2 = 0
route_B4_P3 = 0
route_P1_D01 = 0
route_P1_D02 = 0
route_P1_D03 = 0
route_P1_D04 = 0
route_P1_D05 = 47
route_P1_D06 = 0
route_P1_D07 = 0
route_P1_D08 = 0
route_P1_D09 = 0
route_P1_D10 = 41
route_P1_D11 = 0
route_P1_D12 = 0
route_P1_D13 = 0
route_P1_D14 = 0
route_P1_D15 = 41
route_P2_D01 = 48
route_P2_D02 = 0
route_P2_D03 = 0
route_P2_D04 = 0
route_P2_D05 = 0
route_P2_D06 = 0
route_P2_D07 = 64
route_P2_D08 = 93
route_P2_D09 = 0
route_P2_D10 = 0
route_P2_D11 = 0
route_P2_D12 = 0
route_P2_D13 = 0
route_P2_D14 = 70
route_P2_D15 = 0
route_P3_D01 = 0
route_P3_D02 = 84
route_P3_D03 = 64
route_P3_D04 = 106
route_P3_D05 = 0
route_P3_D06 = 57
route_P3_D07 = 0
route_P3_D08 = 0
route_P3_D09 = 74
route_P3_D10 = 0
route_P3_D11 = 61
route_P3_D12 = 42
route_P3_D13 = 57
route_P3_D14 = 0
route_P3_D15 = 0


In [20]:
B1_output = 0
B2_output = 0
B3_output = 0
B4_output = 0
P1_input = 0
P2_input = 0
P3_input = 0
P1_output = 0
P2_output = 0
P3_output = 0

In [21]:
for v in prob.variables():
    if re.search("route_B1_..",v.name):
        B1_output += round(v.varValue)
    if re.search("route_B2_..",v.name):
        B2_output += round(v.varValue)
    if re.search("route_B3_..",v.name):
        B3_output += round(v.varValue)
    if re.search("route_B4_..",v.name):
        B4_output += round(v.varValue)
    if re.search("route_.._P1",v.name):
        P1_input += round(v.varValue)
    if re.search("route_.._P2",v.name):
        P2_input += round(v.varValue)
    if re.search("route_.._P3",v.name):
        P3_input += round(v.varValue)
    if re.search("route_P1_...",v.name):
        P1_output += round(v.varValue)
    if re.search("route_P2_...",v.name):
        P2_output += round(v.varValue)
    if re.search("route_P3_...",v.name):
        P3_output += round(v.varValue)
total_brewing_output = B1_output + B2_output + B3_output  + B4_output
total_packaging_input = P1_input + P2_input + P3_input
total_packaging_output = P1_output + P2_output + P3_output

In [22]:
print("B1_output:",B1_output)
print("B2_output:",B2_output)
print("B3_output:",B3_output)
print("B4_output:",B4_output)

B1_output: 275
B2_output: 29
B3_output: 545
B4_output: 100


In [23]:
print("P1_input:",P1_input)
print("P2_input:",P2_input)
print("P3_input:",P3_input)
print("P1_output:",P1_output)
print("P2_output:",P2_output)
print("P3_output:",P3_output)
print("total_brewing_output:", total_brewing_output)
print("total_packaging_input:", total_packaging_input)
print("total_packaging_output:", total_packaging_output)
print("total demand:", total_demand)

P1_input: 129
P2_input: 275
P3_input: 545
P1_output: 129
P2_output: 275
P3_output: 545
total_brewing_output: 949
total_packaging_input: 949
total_packaging_output: 949
total demand: 949


In [24]:
print("Total Shipping Costs = ", round(pulp.value(prob.objective)))

Total Shipping Costs =  2342
